In [ ]:
import pandas   as pd
import polars   as pl
import numpy    as np
import networkx as nx
import time
import sys
sys.path.insert(1, '../framework')
from racetrack import *
rt = RACETrack()


_figures_out_initial_filter_ = '''
g_nx = rt.createNetworkXGraph(df, [('sbj','obj','vrb')])
print(f'{len(g_nx)=} | {len(g_nx.edges)=}')
to_keep = set(nx.shortest_path(g_nx, _src_, _dst_))
_path_  = nx.shortest_path(g_nx, _src_, _dst_)
for i in range(len(_path_)-1): 
    g_nx.remove_edge(_path_[i],_path_[i+1])
print(f'{len(g_nx)=} | {len(g_nx.edges)=} | {len(to_keep)=}')
for n0 in _path_:
    for n1 in _path_:
        if n0 == n1: continue
        try:
            _p_ = nx.shortest_path(g_nx, n0, n1)
            to_keep = to_keep | set(_p_)
        except:
            pass
print(f'{len(to_keep)=}')
'''

_filter_op_ = '''
_filter_ = {467458, 2983428, 352774, 5190151, 956937, 1873418, 1873420, 610830, 4527122, 2225684, 3892244, 5252128, 3370535, 3588136, 
            6330927, 233519, 1389103, 1461815, 324153, 6438969, 1891900, 2176575, 1937990, 2799176, 120910, 245839, 3163219, 7760468, 
            770644, 116822, 649307, 264295, 335976, 335977, 329840, 7055472, 1860218, 492671, 5746830, 1862294, 499866, 407200, 121508, 
            501419, 6417068, 5052588, 2226863, 4174018, 1621189, 467663, 6429909, 406744, 226522, 490203, 410845, 214749, 6302444, 
            1391343, 1369338, 611067, 3402499, 6714118, 212231, 503566, 215321, 6438681, 6331163, 6809887, 3147039, 603944, 5917483, 
            1583412, 5259071, 249665, 5902660, 6729546, 2452812, 439628, 256336, 4828499, 214869, 2642781, 2390373, 316774, 324089, 
            6729582, 927087, 5343089, 7745394, 1199987, 631157, 497526, 496503, 645505, 241539, 3066244, 514437, 6646662, 2271624, 
            601481, 742283, 2223509, 5339543, 406938, 3559324, 4130717, 500126, 514469, 1939878, 1939879, 1713576, 5889449, 1939880, 
            5339561, 5953968, 2444210, 310198, 6327734, 6305720, 241593, 1371586, 214980, 2223559, 4529097, 333771, 8009169, 767961, 
            332252, 4862943, 121318, 3146727, 1193960, 5346281, 7494636, 6156782, 295408, 982000, 5339637, 146425, 1889786, 7760382}

ofi  = rt.ontologyFrameworkInstance(base_filename='../../data/kaggle_imdb_600k/imdb_600k_international_movies')
print(f'Original size: {len(ofi.df_triples)}')
df   = ofi.df_triples.filter((pl.col('sbj').is_in(_filter_)) | (pl.col('obj').is_in(_filter_)))
print(f'Initial Filtered Dataframe: {len(df)}')
print(f'Original Filter: {len(_filter_)}')
for _expansion_ in range(4):
    _filter_expand_by_one_ = set(df['sbj']) | set(df['obj'])
    print(f'{_expansion_} Expanded Filter: {len(_filter_expand_by_one_)}')
    ofi  = rt.ontologyFrameworkInstance(base_filename='../../data/kaggle_imdb_600k/imdb_600k_international_movies')
    df   = ofi.df_triples.filter((pl.col('sbj').is_in(_filter_expand_by_one_)) | (pl.col('obj').is_in(_filter_expand_by_one_)))
    print(f'{_expansion_} Expanded Dataframe: {len(df)}')
'''
#ofi  = rt.ontologyFrameworkInstance(base_filename='../../data/kaggle_imdb_600k/20240519_ontology')
#df   = ofi.df_triples.unique()
#print(f'{df.shape=}')
#g_nx = rt.createNetworkXGraph(df, [('sbj','obj','vrb')]) # full graph ... this takes 45.2s for ~8m records
#nx.shortest_path(g_nx, 1632498, 3901566)
#_src_, _dst_ = 1632498, 3901566

In [ ]:
def linkNodeShortest(df, 
                     relationships,                     # [('src','dst'), ('sbj', 'obj', 'vrb'), ... ]
                     pair,
                     color_by              = None,
                     count_by              = None,
                     count_by_set          = False,
                     use_digraph           = False,
                     max_degree_to_show    = 30,        # annotate any nodes with this degree or higher ... None means no annotations
                     node_color            = None,      # None, "#xxxxxx", dict[node] = "#xxxxxx", "label"
                     node_size             = 'medium',  # N one, 'small', 'medium', 'large', 'vary' (vary by count_by), or a number
                     node_labels           = None,      # dict[node] = "label"
                     link_color            = None,      # None, "#xxxxxx", "relationship", "vary" (vary by color_by)
                     link_size             = 2,         # Number of "vary" (vary by count_by)
                     y_path_gap            = 15,        # Needs to be larger than txt_h
                     x_ins                 = 10,
                     y_ins                 = 10,
                     txt_h                 = 10,
                     draw_labels           = False,
                     label_links           = True,      # if the third tuple is set in the relationship
                     max_label_w           = 64,        # actuall max label height... but we'll stick with convention ... None means no limit
                     w                     = 1024):
    return RTLinkNodeShortest(rt_self=rt, df=df, relationships=relationships, pair=pair, 
                              color_by=color_by, count_by=count_by, count_by_set=count_by_set,
                              use_digraph=use_digraph, max_degree_to_show=max_degree_to_show,
                              node_color=node_color, node_size=node_size, node_labels=node_labels,
                              link_color=link_color, link_size=link_size,
                              y_path_gap=y_path_gap, x_ins=x_ins, y_ins=y_ins, txt_h=txt_h, 
                              draw_labels=draw_labels, label_links=label_links, max_label_w=max_label_w, w=w)

class RTLinkNodeShortest(object):
    def __init__(self, rt_self, **kwargs):
        self.rt_self             = rt_self
        self.df                  = kwargs['df']
        self.relationships       = kwargs['relationships']    # [('fm','to'), (('fm1','fm2'),('to1','to2'))]
        self.pair                = kwargs['pair']             # (src,dst))
        self.color_by            = kwargs['color_by']
        self.count_by            = kwargs['count_by']
        self.count_by_set        = kwargs['count_by_set']
        self.use_digraph         = kwargs['use_digraph']      # use a directed graph
        self.max_degree_to_show  = kwargs['max_degree_to_show']
        self.node_color          = kwargs['node_color']
        self.node_size           = kwargs['node_size']
        self.node_labels         = kwargs['node_labels']
        self.link_color          = kwargs['link_color']
        self.link_size           = kwargs['link_size']
        self.y_path_gap          = kwargs['y_path_gap']
        self.x_ins               = kwargs['x_ins']
        self.y_ins               = kwargs['y_ins']
        self.txt_h               = kwargs['txt_h']
        self.draw_labels         = kwargs['draw_labels']
        self.label_links         = kwargs['label_links']
        self.max_label_w         = kwargs['max_label_w']
        self.w                   = kwargs['w']
        self.nodes_rendered      = set()
        self.time_lu             = {'link_labels':0.0}

        # If either from or to are tuples, concat them together... // could improve a little by ensuring any same tuples are not created more than once
        _ts_ = time.time()
        new_relationships = []
        for i in range(len(self.relationships)):
            _fm_ = self.relationships[i][0]
            if type(_fm_) == list or type(_fm_) == tuple:
                new_fm = f'__fmcat{i}__'
                self.df = self.rt_self.createConcatColumn(self.df, _fm_, new_fm)
                _fm_ = new_fm
            _to_ = self.relationships[i][1]
            if type(_to_) == list or type(_to_) == tuple:
                new_to = '__tocat{i}__'
                self.df = self.rt_self.createConcatColumn(self.df, _to_, new_to)
                _to_ = new_to
            if len(self.relationships[i]) == 2: new_relationships.append((_fm_,_to_))
            else:                               new_relationships.append((_fm_,_to_,self.relationships[i][2]))
        self.relationships = new_relationships
        self.time_lu['concat_columns'] = time.time() - _ts_

        self.node_size_px = 3
        if self.node_size is not None:
            if type(self.node_size) == int or type(self.node_size) == float: self.node_size_px = self.node_size
            elif self.node_size == 'small':  self.node_size_px = 2
            elif self.node_size == 'medium': self.node_size_px = 4
            elif self.node_size == 'large':  self.node_size_px = 6
        
        self.link_size_px = 2
        if self.link_size is not None:
            if type(self.link_size) == int or type(self.link_size) == float: self.link_size_px = self.link_size
            elif self.link_size == 'small':  self.link_size_px = 1
            elif self.link_size == 'medium': self.link_size_px = 2
            elif self.link_size == 'large':  self.link_size_px = 3

    # __linkRelationships__(self, n0, n1) - if the third tuple is set in the relationship
    # ... not as efficient as a group by ... but don't expect to be running this on more than a handful of edges...
    def __linkRelationships__(self, n0, n1):
        _ts_ = time.time()
        results = set()
        for _relationship_ in self.relationships:
            if len(_relationship_) == 3:
                if self.rt_self.isPandas(self.df):
                    _df_ = self.df.query(f'{_relationship_[0]} == @n0 and {_relationship_[1]} == @n1')
                    if len(_df_) > 0: results |= set(_df_[_relationship_[2]])
                    if self.use_digraph == False:
                        _df_ = self.df.query(f'{_relationship_[0]} == @n1 and {_relationship_[1]} == @n0')
                        if len(_df_) > 0: results |= set(_df_[_relationship_[2]])
                elif self.rt_self.isPolars(self.df):
                    _df_ = self.df.filter((pl.col(_relationship_[0]) == n0) & (pl.col(_relationship_[1]) == n1))
                    if len(_df_) > 0: results |= set(_df_[_relationship_[2]])
                    if self.use_digraph == False:
                        _df_ = self.df.filter((pl.col(_relationship_[0]) == n1) & (pl.col(_relationship_[1]) == n0))
                        if len(_df_) > 0: results |= set(_df_[_relationship_[2]])
                else:
                    raise Exception('__linkRelationships__() - only supports pandas and polars')
        self.time_lu['link_labels'] += time.time() - _ts_
        return results

    # def _repr_svg_(self):
    def _repr_svg_(self):
        return self.renderSVG()

    # def renderSVG(self):
    def renderSVG(self):
        self.nodes_rendered = set()
        svg = []

        # Create the original graph, find the shortest path and make a version that removes that path
        _ts_ = time.time()
        g_orig = self.rt_self.createNetworkXGraph(self.df, self.relationships, use_digraph=self.use_digraph)
        p      = nx.shortest_path(g_orig, self.pair[0], self.pair[1])
        g      = g_orig.copy()
        for i in range(len(p)-1): g.remove_edge(p[i],p[i+1])
        self.time_lu['create_graph'] = time.time() - _ts_
        self.g_orig = g

        # Determine the label geometry
        _label_w_ = self.y_ins + self.y_path_gap
        if self.draw_labels:
            for i in range(len(p)):
                if self.node_labels is not None and p[i] in self.node_labels: _node_ = str(self.node_labels[p[i]])
                else:                                                         _node_ = str(p[i])
                _w_       = self.rt_self.textLength(_node_, self.txt_h) + self.y_ins + self.y_path_gap
                _label_w_ = max(_label_w_, _w_)
            if self.max_label_w is not None: _label_w_ = min(_label_w_, self.max_label_w)

        # For all view path indices (VPI's) render a small graph showing what that looks like
        y_base = self.y_ins
        for _vpi_ in range(1, len(p)-1):
            # Geometry
            y_top       = y_base
            y_floors    = max(max(abs(len(p) - _vpi_), abs(_vpi_ - len(p))), _vpi_)
            y_base     += self.y_path_gap*y_floors
            x_path_gap  = (self.w - 2*self.x_ins)/(len(p)-1)
            node_to_xy  = {}

            # Render the base path
            for i in range(len(p)-1):
                n0, n1 = p[i], p[i+1]
                x0 = self.x_ins + x_path_gap*i
                node_to_xy[n0] = (x0, y_base)
                x1 = self.x_ins + x_path_gap*(i+1)
                node_to_xy[n1] = (x1, y_base)
                _color_ = self.rt_self.co_mgr.getTVColor('data','default')
                if (self.draw_labels and self.label_links) or self.link_color == 'relationship':
                    link_relationships = self.__linkRelationships__(n0, n1)
                    if   len(link_relationships) >  1: _str_ = '*'
                    elif len(link_relationships) == 1:
                        _pop_ = link_relationships.pop() 
                        _str_ = str(_pop_)
                        if self.link_color == 'relationship': _color_ = self.rt_self.co_mgr.getColor(_pop_)
                    else:                              _str_ = None
                    if _str_ is not None and self.draw_labels and self.label_links: 
                        svg.append(self.rt_self.svgText(self.rt_self.cropText(_str_, self.txt_h, x1-x0), (x0+x1)/2, y_base-2, self.txt_h, anchor='middle'))
                svg.append(f'<line x1="{x0}" y1="{y_base}" x2="{x1}" y2="{y_base}" stroke="{_color_}" stroke-width="{self.link_size_px}" />')

            # Render the base path nodes
            for _node_ in node_to_xy:
                _color_ = self.rt_self.co_mgr.getTVColor('data','default')
                if self.node_color is not None:
                    if   self.node_color.startswith('#') and len(self.node_color) == 7: _color_ = self.node_color
                    elif self.node_color == 'label':
                        if self.node_labels is not None and _node_ in self.node_labels: _color_ = self.rt_self.co_mgr.getColor(_node_)                         
                        else:                                                           _color_ = self.rt_self.co_mgr.getColor(_node_)
                self.nodes_rendered.add(_node_)
                svg.append(f'<circle cx="{node_to_xy[_node_][0]}" cy="{node_to_xy[_node_][1]}" r="{self.node_size_px}" stroke="{_color_}" fill="{_color_}" stroke-width="1" />')

                # Max Degree Annotation
                if self.max_degree_to_show is not None and self.g_orig.degree[_node_] >= self.max_degree_to_show:
                    _color_ = self.rt_self.co_mgr.getTVColor('axis','major')
                    svg.append(f'<circle cx="{node_to_xy[_node_][0]}" cy="{node_to_xy[_node_][1]}" r="{self.node_size_px+2}" stroke="{_color_}" fill="none" stroke-width="2" />')
                
                # Render the node labels
                if self.draw_labels:
                    if self.node_labels is not None and _node_ in self.node_labels: _node_label_ = str(self.node_labels[_node_])
                    else:                                                           _node_label_ = str(_node_)
                    _cropped_ = self.rt_self.cropText(str(_node_label_), self.txt_h, _label_w_-self.y_ins)
                    svg.append(self.rt_self.svgText(_cropped_, node_to_xy[_node_][0]-self.txt_h/2, node_to_xy[_node_][1]+self.txt_h, self.txt_h, anchor='start', rotation=90))

            # Bottom of this specific graph        
            y_bot = y_base + _label_w_

            # Render the alternative paths if the baseline path wasn't in existence
            if _vpi_ > 0 and _vpi_ < len(p)-1:
                node_center = p[_vpi_]
                svg.append(f'<line x1="{node_to_xy[node_center][0]}" y1="{y_top}" x2="{node_to_xy[node_center][0]}" y2="{y_base}" stroke="gray" stroke-width="0.5" stroke-dasharray="1 5 1" />')
            offset = 1
            while offset < len(p):
                for _side_ in ['backward', 'forward']:
                    _my_txt_x_offset_ = -4    if _side_ == 'backward' else 4
                    _my_anchor_       = 'end' if _side_ == 'backward' else 'start'
                    j = _vpi_ - offset if _side_ == 'backward' else _vpi_ + offset                    
                    y = y_base - self.y_path_gap*offset
                    if j >= 0 and j <= len(p)-1:
                        try:    pp = nx.shortest_path(g, p[j],p[_vpi_]) if _side_ == 'backward' else nx.shortest_path(g, p[_vpi_],p[j])
                        except: pp = None
                        if pp is not None:
                            x0, x1 = node_to_xy[pp[0]][0]+x_path_gap/4, node_to_xy[pp[-1]][0]-x_path_gap/4
                            svg.append(f'<line x1="{x0}" y1="{y}" x2="{x1}" y2="{y}" stroke="gray" stroke-width="0.5" />')
                            svg.append(f'<line x1="{x0}" y1="{y}" x2="{node_to_xy[pp[0]][0]}" y2="{node_to_xy[pp[0]][1]}" stroke="gray" stroke-width="0.5" />')
                            svg.append(f'<line x1="{x1}" y1="{y}" x2="{node_to_xy[pp[-1]][0]}" y2="{node_to_xy[pp[-1]][1]}" stroke="gray" stroke-width="0.5" />')
                            svg.append(self.rt_self.svgText(f'{len(pp)}', node_to_xy[p[_vpi_]][0] + _my_txt_x_offset_, y + self.txt_h/2, self.txt_h, anchor=_my_anchor_))
                            if len(pp) > 3:
                                my_x_path_gap = (x1-x0)/(len(pp)-3)
                                for k in range(1, len(pp)-1):
                                    _color_ = self.rt_self.co_mgr.getTVColor('data','default')
                                    if self.node_color is not None:
                                        if   self.node_color.startswith('#') and len(self.node_color) == 7: _color_ = self.node_color
                                        elif self.node_color == 'label':
                                            if self.node_labels is not None and pp[k] in self.node_labels: _color_ = self.rt_self.co_mgr.getColor(pp[k])
                                            else:                                                           _color_ = self.rt_self.co_mgr.getColor(pp[k])
                                    self.nodes_rendered.add(pp[k])
                                    svg.append(f'<circle cx="{x0+(k-1)*my_x_path_gap}" cy="{y}" r="{self.node_size_px}" stroke="{_color_}" fill="{_color_}" stroke-width="0.5" />')

                                    # Max Degree Annotation
                                    if self.max_degree_to_show is not None and self.g_orig.degree[pp[k]] >= self.max_degree_to_show:
                                        _color_ = self.rt_self.co_mgr.getTVColor('axis','major')
                                        svg.append(f'<circle cx="{x0+(k-1)*my_x_path_gap}" cy="{y}" r="{self.node_size_px+2}" stroke="{_color_}" fill="none" stroke-width="2" />')
                            else:
                                _color_ = self.rt_self.co_mgr.getTVColor('data','default')
                                if self.node_color is not None:
                                    if   self.node_color.startswith('#') and len(self.node_color) == 7: _color_ = self.node_color
                                    elif self.node_color == 'label':
                                        if self.node_labels is not None and pp[0] in self.node_labels: _color_ = self.rt_self.co_mgr.getColor(pp[0])
                                        else:                                                          _color_ = self.rt_self.co_mgr.getColor(pp[0])
                                self.nodes_rendered.add(pp[0])
                                svg.append(f'<circle cx="{x0+x_path_gap/2}" cy="{y}" r="{self.node_size_px}" stroke="{_color_}" fill="{_color_}" stroke-width="0.5" />')

                                # Max Degree Annotation
                                if self.max_degree_to_show is not None and self.g_orig.degree[pp[0]] >= self.max_degree_to_show:
                                    _color_ = self.rt_self.co_mgr.getTVColor('axis','major')
                                    svg.append(f'<circle cx="{x0+x_path_gap/2}" cy="{y}" r="{self.node_size_px+2}" stroke="{_color_}" fill="none" stroke-width="2" />')
                        else:
                            svg.append(self.rt_self.svgText('∅', node_to_xy[p[_vpi_]][0] + _my_txt_x_offset_, y + self.txt_h/2, self.txt_h, anchor=_my_anchor_))
                offset += 1
            y_base = y_bot+self.y_path_gap
        y_base += self.y_ins
        svg.insert(0, f'<svg width="{self.w}" height="{y_base}">')
        svg.append('</svg>')
        return ''.join(svg)
df = pl.DataFrame({'sbj':[1, 2, 3, 4, 5, 6,  2,  20, 21, 22, 23, 1,  11, 12, 13, 14, 15, 16, 17, 5,  50, 50],
                   'obj':[2, 3, 4, 5, 6, 7,  20, 21, 22, 23, 4,  11, 12, 13, 14, 15, 16, 17, 5,  50, 4,  4],
                   'vrb':[1, 1, 1, 1, 1, 1,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  5,  5,  6]})
_src_, _dst_ = 1, 7
lns = linkNodeShortest(df, relationships=[('sbj','obj','vrb')], pair=(_src_,_dst_), max_degree_to_show=2,
                       link_color='relationship', link_size='large',
                       node_labels={2:'long string goes here'}, node_color='label', draw_labels=True, max_label_w=64)
lns

In [ ]:
lns.nodes_rendered

In [ ]:
ofi  = rt.ontologyFrameworkInstance(base_filename='../../data/kaggle_imdb_600k/20240519_ontology')
df   = ofi.df_triples.unique()
print(f'{df.shape=}')
#g_nx = rt.createNetworkXGraph(df, [('sbj','obj','vrb')]) # full graph ... this takes 45.2s for ~8m records
#nx.shortest_path(g_nx, 1632498, 3901566)
_src_, _dst_ = 1632498, 3901566
lns = linkNodeShortest(df, relationships=[('sbj','obj','vrb')], pair=(_src_,_dst_), draw_labels=True, node_color='label')
lns

In [ ]:
lns.nodes_rendered

In [ ]:
g = nx.Graph()
g.add_edge('a','b'), g.add_edge('a','c'), g.add_edge('a','d'), g.add_edge('d','e'), g.add_edge('f','d')
g.degree['a'], g.degree['b'], g.degree['c'], g.degree['d']

In [ ]:
lns.time_lu